In [45]:
from datetime import datetime
from langchain_core.tools import tool
from pymongo import MongoClient
import os
from typing import List, Optional, Tuple
from langchain.pydantic_v1 import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain.globals import set_verbose, set_debug
from langchain_openai import AzureOpenAIEmbeddings, AzureChatOpenAI
from langchain_mongodb import MongoDBAtlasVectorSearch
from langchain.agents import AgentExecutor, create_tool_calling_agent

In [3]:
set_verbose(True)
set_debug(True)


In [4]:

embeddings = AzureOpenAIEmbeddings(
    azure_deployment="embedding2",
    openai_api_version="2023-05-15",
)
llm = AzureChatOpenAI(
    azure_deployment="gpt4",
    openai_api_version="2023-05-15",
)

llm.invoke("hello")

[llm/start] [llm:AzureChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: hello"
  ]
}
[llm/end] [llm:AzureChatOpenAI] [2.55s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Hello! How can I assist you today?",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null,
          "content_filter_results": {}
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": "Hello! How can I assist you today?",
            "response_metadata": {
              "token_usage": {
                "completion_tokens": 9,
                "prompt_tokens": 8,
                "total_tokens": 17
              },
              "model_name": "gpt-4",
              "system_fingerprint": "fp_2f57f81c

AIMessage(content='Hello! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 8, 'total_tokens': 17}, 'model_name': 'gpt-4', 'system_fingerprint': 'fp_2f57f81c11', 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {}}, id='run-d5b391ef-d3bd-4bd9-b116-d76f6ec04bde-0')

In [7]:

client: MongoClient = MongoClient(CONNECTION_STRING)


In [37]:
def read_index():
    return  MongoDBAtlasVectorSearch(
        client[DB_NAME][COLLECTION_NAME], embeddings, index_name=INDEX_NAME
    )



In [38]:
vector_index = read_index()
print(vector_index.similarity_search_with_score(k= 4,query="give articles talks about Covid"))

[(Document(page_content='link: https://www.huffpost.com/entry/cdc-covid-19-quarantine-social-distancing-rules_n_62f55ed7e4b0526eaee8fe8c,headline CDC Drops Some Quarantine, Screening Recommendations For COVID-19,authors: MIKE STOBBE and COLLIN BINKLEY, AP,category:  U.S. NEWS,short_description:  So long, social distancing.', metadata={'_id': {'$oid': '6650978287b8c22287409dd5'}, 'link': 'https://www.huffpost.com/entry/cdc-covid-19-quarantine-social-distancing-rules_n_62f55ed7e4b0526eaee8fe8c', 'headline': 'CDC Drops Some Quarantine, Screening Recommendations For COVID-19', 'category': 'U.S. NEWS', 'short_description': 'So long, social distancing.', 'authors': 'MIKE STOBBE and COLLIN BINKLEY, AP', 'date': datetime.datetime(2022, 8, 11, 0, 0)}), 0.8958898782730103), (Document(page_content='link: https://www.huffpost.com/entry/cdc-rochelle-walensky-reset_n_62fd1241e4b0f48a02037714,headline CDC Director To Overhaul Agency With COVID Shortcomings In Mind,authors: Mike Stobbe, AP,category:  

In [39]:

class NewsInput(BaseModel):
    category: Optional[str] = Field(
        description="Any particular category that the user wants to finds information for. Here are some examples: "
        +  """{Input:show me articles about food ? category: food} , {Input: is there any articles tagged U.S. News talking about about Covid ? category: U.S. News"""
    )
    authors: Optional[str] = Field(
        description="the Author  Name that wrote articles the user wants to find articles for "
        +"""{Input:give article written by Marry lother? Auther: Marry lother}, {input: is Nina Golgowski have any articles? Author:Nina Golgowski """
    )
    date: Optional[str] = Field(
        description="the  date of an article that the use want to use to filter article, rewrite it format yyyy-MM-ddTHH:mm:ss"
    )
    determination: Optional[str] = Field(
        description="the condition for the date that the user want to filter on ", enum=["before", "after","equal"]
    )
    desc: Optional[str] = Field(
        description="the details and description in the article the user is looking  in the article or contained in the article"
    )

In [40]:
# format the result docummnts for LLM 
def format_docs(docs):
    result=[]
    print(docs)
    for doc in docs:
        result.append( doc[0].page_content )
    return "\n\n".join(result )

In [42]:
@tool(args_schema=NewsInput)
def get_articles(
    category: Optional[str] = None,
    authors: Optional[str] = None,
    date: Optional[str] = None,
desc: Optional[str] = None,
        determination:Optional[str] =None
) -> str:
    "useful for when you need to find relevant information in the news"
    vector_index = read_index()

    condition=''
    if determination =="before":
        condition = "$lte"
    elif determination =="after":
        condition="$gte"
    elif determination == "equal":
        condition="$eq"

    filter ={}
    if category is not None:
        filter["category"]= {"$eq": category.upper()}
    if  authors is not None:
        filter["authors"] = {"$eq": authors}
    if date is not None:
        filter["date"] = {condition:  datetime.fromisoformat(date)}

    return  format_docs(vector_index.similarity_search_with_score(k= 4,query=desc if desc else '', pre_filter = {'$and': [
        filter ] }))

In [43]:
tools = [get_articles]


prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant that finds information about articles "
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [44]:
def _format_chat_history(chat_history: List[Tuple[str, str]]):
    buffer = []
    for human, ai in chat_history:
        buffer.append(HumanMessage(content=human))
        buffer.append(AIMessage(content=ai))
    return buffer


In [49]:
agent = create_tool_calling_agent(llm, tools, prompt)
question={"input": "give me articles written by Elyse Wanshel after 22nd of Sep about Comedy","chat_history":[],"agent_scratchpad":""}

agent_executor = AgentExecutor(agent=agent, tools=tools)

print("Question: ",question)
result=agent_executor.invoke(question)
print("Answer:\n", result['output'])

Question:  {'input': 'give me articles written by Elyse Wanshel after 22nd of Sep about Comedy', 'chat_history': [], 'agent_scratchpad': ''}
[chain/start] [chain:AgentExecutor] Entering Chain run with input:
{
  "input": "give me articles written by Elyse Wanshel after 22nd of Sep about Comedy",
  "chat_history": [],
  "agent_scratchpad": ""
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad>] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agent_scratchpad>] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agent_scratchpad> > chain:RunnableLambda] Entering Chain run with input:
